# TP 6: Clase 7

Utilizando la wecam
 
Implementar un algoritmo utilizando filtros de Haar en cascada que:
- Detecte rostros
- Dentro de los rostros detecte ojos
- Dentro de los rostros detecte sonrisas 

In [1]:
# Se importan las librerías
import cv2 as cv
import time

In [2]:
# Se importan los modelos
face_cascade = cv.CascadeClassifier('tp6_files/haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('tp6_files/haarcascade_eye.xml')
smile_cascade = cv.CascadeClassifier('tp6_files/haarcascade_smile.xml')

In [19]:
# Cargamos la imagen a analizar
#------------------------------
camera_number = 1
camera = cv.VideoCapture(camera_number)

while(1):
    ret, frame = camera.read()
    # Pasamos la imagen a escala de grises
    #-------------------------------------
    grayImage = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    grayImage = cv.equalizeHist(grayImage)

    # Llamamos al clasificador de Haar (AdaBoost)
    #--------------------------------------------
    #faces = face_cascade.detectMultiScale(grayImage, 1.012, 5)
    faces = face_cascade.detectMultiScale(grayImage, 1.1, minNeighbors=3)

    # Recorro las caras encontradas
    #------------------------------
    for (x, y, w, h) in faces:
        # Le dibujamos un rectángulo amarillo
        cv.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 0), 2)
        # Definimos las ROIs en la imagen gris y color
        roi_gray = grayImage[y:y + h, x:x + w] 
        roi_color = frame[y:y + h, x:x + w] 
        # Para cada rostro hallado le buscamos los ojos
        eyes = eye_cascade.detectMultiScale(roi_gray, minNeighbors=5, minSize=(70, 70), maxSize=(140, 140))
        # En los ojos hallados les dibujamos rectángulos
        for (ex, ey, ew, eh) in eyes:
            cv.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)


        # Se hayan las caras y se dibujan rectangulos
        smiles = smile_cascade.detectMultiScale(roi_gray, minNeighbors=15, minSize=(70, 70), maxSize=(140,140))
        for (sx, sy, sw, sh) in smiles:
            cv.rectangle(roi_color, (sx, sy), (sx + sw, sy + sh), (0, 0, 255), 2)
        
        
    cv.imshow('Detección de caras con filtros de Haar en cascada', frame) 
    
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break

cv.destroyAllWindows()
camera.release()